In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t
import moyra as ma
me.mechanics_printing()

In [2]:
# define Degrees of freedom
bending = 3
in_plane_bend = 0
torsion = 2
inner_dofs = bending + in_plane_bend + torsion
DoFs = inner_dofs + 1 + 1
# DoFs = inner_dofs

In [3]:
# generate parameters
p = ma.DynamicModelParameters(DoFs)

#gravity vector
p.DoFs = ma.ModelSymbol(value = DoFs,string = 'DoFs')                     # DoFs
p.g  = ma.ModelSymbol(value = 9.81,string = 'g')                     # gravity
p.g_v = ma.ModelVector(value =[0,0,-1],length=3,string='g_v') # The stationary point

# main geometric properties
p.y_h = ma.ModelSymbol(value = 0.3,string = 'y_h')                  # y position of hinge line along beam axis
p.Lambda = ma.ModelSymbol(value = 0.3,string = 'Lambda')            # Flare Angle

# inner beam properties
p.b_i = ma.ModelSymbol(value = 0.3,string = 'b_i')                  # y position beam end
p.rho_b = ma.ModelSymbol(value = 0.3,string = 'rho_b')              # density per unit area      
p.c_b = ma.ModelSymbol(value = 0.3,string = 'c_b')                  # beam chord
p.EI = ma.ModelSymbol(value = 0.3,string = 'EI')                    # beam flexural rigidity
p.GJ = ma.ModelSymbol(value = 0.3,string = 'GJ')                    # beam torsional rigidity
p.K = ma.ModelSymbol(value = 0.3,string = 'K')                      # beam Cross Coupling

# inner Section mass properties
p.m_s = ma.ModelSymbol(value = 0,string = 'm_s')                    # Sectional Mass
p.I_s = ma.ModelVector(length=3,string='I_s')        # Moments of Inertia for Section 
p.x_s = ma.ModelSymbol(value = 0.3,string = 'x_s')                  # x postion of the CoM of each section 
p.y_s = ma.ModelVector(length=3,string='y_s')        # y position of each of the 3 sections

# # Aileron servo mass properties
# p.m_sv = ma.ModelSymbol(value = 0,string = 'm_sv')                  # Servo Mass
# p.X_sv = ma.ModelMatrix(value =[1,1,0],length=3,string='X_sv')      # Servo x,y,z postion (z ignored)

# inner Hinge mass properties
p.m_h = ma.ModelSymbol(value = 0,string = 'm_h')                    # additional hinge mass section
p.I_h = ma.ModelVector(value =[0,0,0],length=3,string='I_h')        # Moments of inertia of additonal hinge mass
p.X_h = ma.ModelVector(value =[1,1,0],length=3,string='X_h')        # location of addtional hinge mass (z ignored)
p.k_h = ma.ModelSymbol(value = 0,string = 'k_h')                    # Hinge Stiffness
p.d_h = ma.ModelSymbol(value = 0,string = 'd_h')                    # Hinge Damping

# FWT mass properties
p.m_f = ma.ModelSymbol(value = 0,string = 'm_f')                    # FWT mass
p.I_f = ma.ModelVector(value =[0,0,0],length=3,string='I_f')        # FWT moments of inertia
p.X_f = ma.ModelVector(value =[1,1,0],length=3,string='X_f')        # location of FWT CoM relative to hinge semi-chord

# Xb = ma.ModelVector(value =[1,1,0],length=3,string='Xb')
# Xb[0],Xb[1],p.Gamma_0 = sym.symbols('x_0,y_0,Gamma_0')
p.Gamma_0 = sym.symbols('Gamma_0')
Xb = ma.VarVector('Xb',3,1)

In [4]:
## GET INNER WING ELEMENTS
# get shape function
q_f = p.q[1:inner_dofs+1]
S,tau = ma.elements.FlexiElement.ShapeFunctions_OBM_IBN_TO(bending,in_plane_bend,torsion,q_f,Xb[1],Xb[0],0,1,type='taylor')
u = S*sym.Matrix(q_f) + sym.Matrix([Xb[0],Xb[1],0])
body_frame = ma.frames.HomogenousFrame().R_x(p.q[0])
inner_wing_frame = body_frame.Translate(*u)

## create inner wing elements
inner_elements = []
# get beam element
link2_x__int = (Xb[0],-p.c_b*sym.Rational(1,2),p.c_b*sym.Rational(1,2))
link2_y_int = (Xb[1],0,p.b_i)
main_wing_ele = ma.elements.FlexiElement(p.q,inner_wing_frame,p.rho_b,S,link2_x__int,link2_y_int,0,q_f,p.EI,p.GJ,grav_vec = p.g*p.g_v,simplify = False, name="Inner Beam",K=p.K)
inner_elements.append(main_wing_ele)

# get wing section elements
M_s = ma.elements.MassMatrix(p.m_s,*p.I_s)
for i in range(len(p.y_s)):
    tmp_frame = inner_wing_frame.R_y(tau).subs({Xb[0]:p.x_s,Xb[1]:p.y_s[i]})
    inner_elements.append(ma.elements.RigidElement(p.q,tmp_frame,M_s,grav_vec = p.g*p.g_v, simplify = False, name=f"Wing Section {i+1}"))
    
# get hinge mass element
M = ma.elements.MassMatrix(p.m_h,*p.I_h)
tmp_frame = inner_wing_frame.R_y(tau).subs({Xb[0]:p.X_h[0],Xb[1]:p.X_h[1]})
inner_elements.append(ma.elements.RigidElement(p.q,tmp_frame,M,grav_vec = p.g*p.g_v, simplify = False, name="Hinge Mass Element"))

# # get servo mass element
# M = ma.elements.MassMatrix(p.m_sv,0,0,0)
# tmp_frame = inner_wing_frame.subs({Xb[0]:p.X_sv[0],Xb[1]:p.X_sv[1]})
# inner_elements.append(ma.elements.RigidElement(p.q,tmp_frame,M,grav_vec = p.g*p.g_v, simplify = False, name="Servo Mass Element"))

In [5]:
## GET FWT ELEMENT

# get angles at end of inner beam
dz_dy = u.subs({Xb[0]:0}).diff(Xb[1]).subs({Xb[1]:p.y_h})
dz_dx = u.subs({Xb[1]:p.y_h}).diff(Xb[0]).subs({Xb[0]:0})
Ry = sym.Function('Ry')(t)
Rx = sym.Function('Rx')(t)

# get fwt frames
# hinge_rot = ma.frames.HomogenousFrame().R_y(dz_dx[2]/dz_dx[0]).R_z(p.Lambda).simplify().A
# fold_rot = ma.frames.HomogenousFrame().R_x(dz_dy[2]/dz_dy[1]-p.q[-1]).R_z(-p.Lambda).simplify().A

# hinge_rot = ma.frames.HomogenousFrame().R_y(-dz_dx[2]/dz_dx[0]).R_x(p.Gamma_0).R_z(p.Lambda).simplify().A
hinge_rot = ma.frames.HomogenousFrame().R_y(Ry).R_x(Rx).R_z(p.Lambda).simplify().A
fold_rot = ma.frames.HomogenousFrame().R_x(-p.q[-1]).R_z(-p.Lambda).simplify().A

# hinge_rot = ma.frames.HomogenousFrame().R_y_small(Ry,ExpandCosine=False).R_x_small(Rx,ExpandCosine=False).R_z(p.Lambda).simplify().A
# fold_rot = ma.frames.HomogenousFrame().R_x(-p.q[-1]).R_z(-p.Lambda).simplify().A

hinge_frame = inner_wing_frame.subs({Xb[1]:p.y_h,Xb[0]:0}).Rotate(hinge_rot).simplify()
fwt_base_frame = hinge_frame.Rotate(fold_rot).simplify()
fwt_frame = fwt_base_frame.Translate(*p.X_f)

#create fwt main elements
fwt_elements = []
M = ma.elements.MassMatrix(p.m_f,p.I_f[0],p.I_f[1],p.I_f[2])
# M = ma.elements.MassMatrix(p.m_f,0,0,0)
fwt_elements.append(ma.elements.RigidElement(p.q,fwt_frame,M,grav_vec = p.g*p.g_v, simplify = False,name="FWT"))

# get hinge spring and damper elements
fwt_elements.append(ma.elements.Spring(p.q,p.q[-1],p.k_h,name="Hinge Spring"))
fwt_elements.append(ma.elements.Damper(p.q,p.q[-1],p.d_h,name="Hinge Damper"))


In [6]:
# sm = ma.SymbolicModel.FromElementsAndForces(p.q,[*inner_elements,*fwt_elements])
# Xi = ma.VarVector('Xi',3)


# subs = {p.Gamma_0:dz_dy[2]/dz_dy[1]}

subs = {Rx.diff(t):(dz_dy[2]/dz_dy[1]).diff(t),Ry.diff(t):(-dz_dx[2]/dz_dx[0]).diff(t),
          Rx:dz_dy[2]/dz_dy[1],Ry:-dz_dx[2]/dz_dx[0]}
# subs = {Rx.diff(t):0,Ry.diff(t):0,
#           Rx:dz_dy[2]/dz_dy[1],Ry:-dz_dx[2]/dz_dx[0]}

sm = ma.SymbolicModel.FromElementsAndForces(p.q,[*inner_elements,*fwt_elements]).subs(subs)
# sm = ma.SymbolicModel.FromElementsAndForces(p.q,[inner_elements[0]]).subs(subs)
fwt_frame = fwt_frame.subs(subs)
fwt_base_frame = fwt_base_frame.subs(subs)
# %lprun -f ma.SymbolicModel.FromElementsAndForces ma.SymbolicModel.FromElementsAndForces(p.q,[main_wing_ele,fwt_ele])

Generating EoM for Element 1 out of 8 - Inner Beam:FlexiElement
Generating EoM for Element 2 out of 8 - Wing Section 1:RigidElement
Generating EoM for Element 3 out of 8 - Wing Section 2:RigidElement
Generating EoM for Element 4 out of 8 - Wing Section 3:RigidElement
Generating EoM for Element 5 out of 8 - Hinge Mass Element:RigidElement
Generating EoM for Element 6 out of 8 - FWT:RigidElement
Generating EoM for Element 7 out of 8 - Hinge Spring:Spring
Generating EoM for Element 8 out of 8 - Hinge Damper:Damper


In [7]:
funcs = []
Xi = ma.VarVector('Xi',3,1)
funcs.append(('get_pos_inner_wing',inner_wing_frame.transform_point([0]*3)))
funcs.append(('get_pos_fwt',fwt_frame.transform_point(Xi)))
funcs.append(('get_fwt_V_b',fwt_base_frame.A.T*fwt_base_frame.transform_point(Xi).diff(t)))
funcs.append(('get_fwt_V_global',fwt_base_frame.transform_point(Xi).diff(t)))
funcs.append(('get_pos_global2fwt',fwt_base_frame.transform_global_point(Xi)))
funcs.append(('get_pos_fwt2global',fwt_base_frame.transform_point(Xi)))
funcs.append(('get_fwt_V_b_body',body_frame.A.T*fwt_base_frame.transform_point(Xi).diff(t)))
funcs.append(('get_hda',dz_dy[2]/dz_dy[1]))
funcs.append(('get_fwt_A',fwt_frame.A))
funcs.append(('get_inner_A',inner_wing_frame.A))
funcs.append(('get_body_A',body_frame.A))

funcs.append(('get_inner_V_b',inner_wing_frame.A.T*inner_wing_frame.R.diff(t)))

In [8]:
fwt_normal = fwt_frame.transform_vector([0,0,1])
fwt_hinge_vector = fwt_frame.transform_vector([sym.cos(p.Lambda),sym.sin(p.Lambda),0])
surf = inner_wing_frame.transform_point([0]*3)
v = ma.Wedge(surf.diff(Xb[0]))*surf.diff(Xb[1])
hinge_normal = sym.simplify(v.subs({Xb[0]:0,Xb[1]:p.y_h}))
surf = inner_wing_frame.transform_point([0]*3)
funcs.append(('get_surf_x',surf.subs({Xb[0]:1})-surf.subs({Xb[0]:0})))
funcs.append(('get_surf_z',sym.simplify(v)))
funcs.append(('get_fold',p.q[-1]))
funcs.append(('get_fold_dt',p.qd[-1]))
funcs.append(('get_hinge_normal',hinge_normal))
funcs.append(('get_fwt_normal',fwt_normal))
funcs.append(('get_fwt_hinge_vector',fwt_hinge_vector))

In [9]:
# generate generic force symbols
F_x,F_y,F_z,M_x,M_y,M_z = sym.symbols('F_x,F_y,F_z,M_x,M_y,M_z')
W_n = sym.Matrix([F_x,F_y,F_z,M_x,M_y,M_z])
W = ma.VarVector('W',6,1)
#create function for an arbitary force on FWT
fwt_Q = fwt_base_frame.Translate(*Xi).BodyJacobian(p.q).T*W_n
fwt_Q = me.msubs(fwt_Q,{**{W_n[i]:W[i,0] for i in range(6)}})
funcs.append(('get_fwt_Q',me.msubs(fwt_Q,{p.q[0]:0,p.qd[0]:0}))) # roll does not affect the generalised force

#create function for an arbitary force on Main Wing
MW_Q = inner_wing_frame.BodyJacobian(p.q).T*W_n
MW_Q = me.msubs(MW_Q,{**{W_n[i]:W[i,0] for i in range(6)}})
funcs.append(('get_inner_Q',me.msubs(MW_Q,{p.q[0]:0,p.qd[0]:0}))) # roll does not affect the generalised force

In [10]:
base_dir = "C:/git/roll_flexibility/src_matlab"
class_name = f'Roll_B{bending}T{torsion}_FWT_RC'
base_class = "mbd.BaseRC"
sm.to_matlab_class(p,base_dir,class_name,base_class,additional_funcs=funcs)

Unknown variable Xb found in function get_pos_inner_wing. It will be added to the function signature.
Unknown variable Xi found in function get_pos_fwt. It will be added to the function signature.
Unknown variable Xi found in function get_fwt_V_b. It will be added to the function signature.
Unknown variable Xi found in function get_fwt_V_global. It will be added to the function signature.
Unknown variable Xi found in function get_pos_global2fwt. It will be added to the function signature.
Unknown variable Xi found in function get_pos_fwt2global. It will be added to the function signature.
Unknown variable Xi found in function get_fwt_V_b_body. It will be added to the function signature.
Unknown variable Xb found in function get_inner_V_b. It will be added to the function signature.
Unknown variable Xb found in function get_surf_x. It will be added to the function signature.
Unknown variable Xb found in function get_surf_z. It will be added to the function signature.
Unknown variable Xi